## Scrape URLs

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
from datetime import datetime
import pandas as pd
import json
import pymongo
from selenium import webdriver

In [ ]:
maxLen = 200

In [ ]:
def create_ListofDict(u,h,d,t):
    listofdict = []
    for c,lx in enumerate(u,0):
        info = {
            "timestamp": t,
            "URL":    lx,
            "headline": h[c],
            "newsdate":  d[c]
        }
        listofdict.append(info)
    return listofdict

In [ ]:
def convertTHMonth(monthString):
    switcher = {
        "มกราคม": 1,
        "กุมภาพันธ์": 2,
        "มีนาคม": 3,
        "เมษายน":  4,
        "พฤษภาคม": 5,
        "มิถุนายน": 6,
        "กรกฎาคม": 7,
        "สิงหาคม": 8,
        "กันยายน": 9,
        "ตุลาคม": 10,
        "พฤศจิกายน": 11,
        "ธันวาคม": 12}
    return switcher.get(monthString)

In [ ]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["BADS7101"]
q_c_urlList = mydb["URL_LIST_CRIME"]
q_p_urlList = mydb["URL_LIST_POLITIC"]
q_e_urlList = mydb["URL_LIST_ENTER"]

### Scrape URL

In [ ]:
urls_crime = 'https://www.dailynews.co.th/crime?page='
urls_political = 'https://www.dailynews.co.th/politics?page='
urls_entertain = 'https://www.dailynews.co.th/entertainment?page='

In [ ]:
q_c_urlList.delete_many({})

for e in range(1,maxLen):
    today = datetime.now()
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(urls_crime+str(e))
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    urlList,dtList,hList = [],[],[]
    
    for div in soup.find_all("div", class_=lambda value: value and value=="left"):
        for a in div.find_all("a", class_=lambda value: value and value=="media", href=True):
            urlList.append("https://www.dailynews.co.th"+str(a['href']).strip())
            for s in a.find_all("span", class_=lambda value: value and value=="media-date"):
                dstr = s.text.strip().split()
                dt = str(dstr[1])+"/"+str(convertTHMonth(dstr[2]))+"/"+str(int(dstr[3])-543)+" "+str(dstr[5]).replace('.',':')
                dtList.append(datetime.strptime(dt, '%d/%m/%Y %H:%M'))
            for h in a.find_all("h3", class_=lambda value: value and value=="media-heading"):
                hList.append(str(h.text.strip()))
    process_data = create_ListofDict(urlList,hList,dtList,today)
    q_c_urlList.insert_many(process_data)
    time.sleep(2)
    driver.quit()

In [ ]:
#q_p_urlList.delete_many({})

for e in range(207,421):
    today = datetime.now()
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(urls_political+str(e))
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    urlList,dtList,hList = [],[],[]
    
    for div in soup.find_all("div", class_=lambda value: value and value=="left"):
        for a in div.find_all("a", class_=lambda value: value and value=="media", href=True):
            urlList.append("https://www.dailynews.co.th"+str(a['href']).strip())
            for s in a.find_all("span", class_=lambda value: value and value=="media-date"):
                dstr = s.text.strip().split()
                dt = str(dstr[1])+"/"+str(convertTHMonth(dstr[2]))+"/"+str(int(dstr[3])-543)+" "+str(dstr[5]).replace('.',':')
                dtList.append(datetime.strptime(dt, '%d/%m/%Y %H:%M'))
            for h in a.find_all("h3", class_=lambda value: value and value=="media-heading"):
                hList.append(str(h.text.strip()))
    process_data = create_ListofDict(urlList,hList,dtList,today)
    q_p_urlList.insert_many(process_data)
    time.sleep(2)
    driver.quit()

In [ ]:
q_e_urlList.delete_many({})

for e in range(1,maxLen):
    today = datetime.now()
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(urls_entertain+str(e))
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    urlList,dtList,hList = [],[],[]
    
    for div in soup.find_all("div", class_=lambda value: value and value=="left"):
        for a in div.find_all("a", class_=lambda value: value and value=="media", href=True):
            urlList.append("https://www.dailynews.co.th"+str(a['href']).strip())
            for s in a.find_all("span", class_=lambda value: value and value=="media-date"):
                dstr = s.text.strip().split()
                dt = str(dstr[1])+"/"+str(convertTHMonth(dstr[2]))+"/"+str(int(dstr[3])-543)+" "+str(dstr[5]).replace('.',':')
                dtList.append(datetime.strptime(dt, '%d/%m/%Y %H:%M'))
            for h in a.find_all("h3", class_=lambda value: value and value=="media-heading"):
                hList.append(str(h.text.strip()))
    process_data = create_ListofDict(urlList,hList,dtList,today)
    q_e_urlList.insert_many(process_data)
    time.sleep(2)
    driver.quit()